In [1]:
from selenium import webdriver
import pandas as pd
import time
from bs4 import BeautifulSoup
import re
from fontTools.ttLib import TTFont
import requests
from lxml import html

In [4]:
car_lst={'沃尔沃S90':'http://newcar.xcar.com.cn/3382/#clicklog=125311', '宝马5系':'http://newcar.xcar.com.cn/22/#clicklog=125311',
        '雷克萨斯ES':'http://newcar.xcar.com.cn/389/#clicklog=124582', '奥迪Q5L': 'http://newcar.xcar.com.cn/3865/#clicklog=124582',
        '凯迪拉克CT5':'http://newcar.xcar.com.cn/4628/#clicklog=125311', '奥迪A4L':'http://newcar.xcar.com.cn/553/#clicklog=125311',
        '宝马3系':'http://newcar.xcar.com.cn/21/#clicklog=125311','凯迪拉克CT5':'http://newcar.xcar.com.cn/4628/#clicklog=125311',
         '奥迪A4L':'http://newcar.xcar.com.cn/553/#clicklog=125311','奔驰C':'http://newcar.xcar.com.cn/39/', 
         '奔驰E': 'http://newcar.xcar.com.cn/40/'}

In [6]:
for name,url in car_lst.items():
    driver = webdriver.Firefox()
    driver.get(url)
    driver.find_element_by_link_text('口碑').click()
    time.sleep(2)
    basic_score_raw = driver.find_element_by_class_name('column').text.split()
    basic_score = {basic_score_raw[i*2]:basic_score_raw[i*2+1] for i in range(int(len(basic_score_raw)/2))}
    post_dict = {}
    for page in range(50):
        body = driver.find_elements_by_class_name('alibi_wrap')[1]
        imp_lst = body.find_elements_by_class_name('home_list.clearfix')
        for imp in imp_lst:
            try:
                purch_info = imp.find_element_by_class_name('list_infor')
                info_lst = purch_info.find_elements_by_class_name('clearfix')
                info_dict = {info_lst[i].text.split('\n')[0]: info_lst[i].text.split('\n')[1] for i in range(len(info_lst))}
            except:
                info_dict = {}
            try:
                info_dict['publish time'] = imp.find_element_by_class_name('publish').text.split()[1]
            except:
                info_dict['publish time'] = ''
            try:
                score_lst = imp.find_element_by_class_name('score_list').text.split()
                score = {score_lst[i*2]:score_lst[i*2+1] for i in range(int(len(score_lst)/2))}
                info_dict['score'] = score
            except:
                info_dict['score'] = ''
            try:
                purch_purp = imp.find_element_by_class_name('purpose.clearfix').text.split()
                info_dict['purch_purp'] = purch_purp
            except:
                info_dict['purch_purp'] = ''
            try:
                info_dict['simple_review'] = imp.find_element_by_class_name('jia_text').text
            except:
                info_dict['simple_review'] = ''
            try:
                imp.find_element_by_link_text('展开全部').click()
                review = imp.find_element_by_class_name('review_post').text
                info_dict['review'] = review
            except:
                try:
                    review = imp.find_element_by_class_name('review_post').text
                    info_dict['review'] = review
                except:
                    info_dict['review'] = ''
            post_dict[len(post_dict)] = info_dict
            time.sleep(5)
        try:
            driver.find_element_by_class_name('page_down').click()
        except:
            break
        time.sleep(5)
    driver.close()
    post_df = pd.DataFrame(post_dict).T
    post_df.to_csv(name+'_XCar.csv',encoding='utf_8_sig')